In [1]:
from collections import Counter, defaultdict
import pandas as pd
import numpy as np

In [2]:
teams = pd.read_csv("../../data/processed/teams.csv")
teams["team_dashed"] = teams.apply(lambda row: (str(row.city) + "-" + str(row["name"])).lower().replace(" ", "-"), axis=1)
teams["city_name"] = teams.apply(lambda row: str(row.city) + " " + str(row["name"]), axis=1)
team_dashed_to_team_id = teams.set_index('team_dashed')['team_id'].to_dict()
city_name_to_team_id = teams.set_index('city_name')['team_id'].to_dict()
city_to_city_name = teams.set_index('city')['city_name'].to_dict()

old_to_new_id = {}
for index, row in teams.iterrows():
    for old_id in eval(row.old_ids):
        old_to_new_id[old_id] = row.team_id

In [3]:
def opp_to_team_id(row):
    d = {
        "LA Chargers": "Los Angeles Chargers",
        "LA Rams": "Los Angeles Rams",
        "N.Y. Giants": "New York Giants",
        "N.Y. Jets": "New York Jets",
        "St. Louis": "Los Angeles Rams",
        "San Diego": "Los Angeles Chargers",
        "Oakland": "Las Vegas Raiders",
    }
    opp = row.Opp
    return city_name_to_team_id[city_to_city_name[opp]] if opp not in d else \
        city_name_to_team_id[d[opp]]

def clean_week(row):
    d = {
        'Conference Championships': 20,
        'Divisional Playoffs': 19,
        'Super Bowl': 21,
        'Wildcard Weekend': 18,
    }
    week = row.Week
    return int(week) if week.isnumeric() else int(d[week] + (1 if row.Year >= 2021 else 0))

def clean_year(row):
    date_month, _, date_year = row.Date.split("/")
    if (date_year == "2023" and str(row.Year) == "2024") or (date_year == "2024" and int(date_month) <= 2):
        return "2023"
    return row.Year

def time_left(row):
    mins, secs = row.Time.split(":")
    return 60 * (60 - 15 * int(row.Quarter)) + 60 * int(mins) + int(secs)

penalties = pd.read_csv("../../data/raw/penalties.csv")
penalties = penalties[penalties.Phase != "ST"]
penalties = penalties[penalties.apply(lambda row: type(row.Time) is str, axis=1)]
penalties["Week"] = penalties.apply(clean_week, axis=1) 
penalties["Phase_Penalty"] = penalties.apply(lambda row: (str(row.Phase) + " " + str(row.Penalty)).replace(" ", "_"), axis=1)
penalties["Team_ID"] = penalties.apply(lambda row: team_dashed_to_team_id[row.Team], axis=1)
penalties["Opp_Team_ID"] = penalties.apply(opp_to_team_id, axis=1)
penalties["Year"] = penalties.apply(clean_year, axis=1)
penalties["Game_ID"] = penalties.apply(lambda row: "_".join([str(row.Year), str(row.Week), str(row.Team_ID), str(row.Opp_Team_ID)],), axis=1)
penalties["Reverse_Game_ID"] = penalties.apply(lambda row: "_".join([str(row.Year), str(row.Week), str(row.Opp_Team_ID), str(row.Team_ID)]), axis=1)
penalties["Time_Left"] = penalties.apply(time_left, axis=1)
penalties

,Penalty,Date,Opp,Player,Pos,Week,Ref Crew,Quarter,Time,Down,...,Home,Phase,Team,Year,Phase_Penalty,Team_ID,Opp_Team_ID,Game_ID,Reverse_Game_ID,Time_Left
0,Offensive Pass Interference,09/13/2009,San Francisco,L.Fitzgerald,WR,1,Don Carey,1,09:43,2,...,Yes,Off,arizona-cardinals,2009,Off_Offensive_Pass_Interference,ARI,SF,2009_1_ARI_SF,2009_1_SF_ARI,3283
1,False Start,09/13/2009,San Francisco,M.Gandy,T,1,Don Carey,1,09:13,2,...,Yes,Off,arizona-cardinals,2009,Off_False_Start,ARI,SF,2009_1_ARI_SF,2009_1_SF_ARI,3253
2,Defensive Offside,09/13/2009,San Francisco,A.Wilson,SS,1,Don Carey,1,07:23,2,...,Yes,Def,arizona-cardinals,2009,Def_Defensive_Offside,ARI,SF,2009_1_ARI_SF,2009_1_SF_ARI,3143
4,Unnecessary Roughness,09/13/2009,San Francisco,A.Wilson,SS,1,Don Carey,1,02:55,1,...,Yes,Def,arizona-cardinals,2009,Def_Unnecessary_Roughness,ARI,SF,2009_1_ARI_SF,2009_1_SF_ARI,2875
5,False Start,09/13/2009,San Francisco,A.Becht,TE,1,Don Carey,1,00:38,1,...,Yes,Off,arizona-cardinals,2009,Off_False_Start,ARI,SF,2009_1_ARI_SF,2009_1_SF_ARI,2738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62658,False Start,12/31/2023,San Francisco,T.Scott,OL,17,Ron Torbert,3,01:05,2,...,Yes,Off,washington-commanders,2023,Off_False_Start,WAS,SF,2023_17_WAS_SF,2023_17_SF_WAS,965
62659,False Start,01/07/2024,Dallas,J.Bates,TE,18,Shawn Smith,2,07:52,3,...,Yes,Off,washington-commanders,2023,Off_False_Start,WAS,DAL,2023_18_WAS_DAL,2023_18_DAL_WAS,2272
62660,False Start,01/07/2024,Dallas,S.Cosmi,OL,18,Shawn Smith,2,07:52,3,...,Yes,Off,washington-commanders,2023,Off_False_Start,WAS,DAL,2023_18_WAS_DAL,2023_18_DAL_WAS,2272
62661,Unnecessary Roughness,01/07/2024,Dallas,C.Barton,LB,18,Shawn Smith,2,06:58,1,...,Yes,Def,washington-commanders,2023,Def_Unnecessary_Roughness,WAS,DAL,2023_18_WAS_DAL,2023_18_DAL_WAS,2218


In [4]:
def clean_game_id(row):
    game_id = row.game_id
    year, week, team1, team2 = game_id.split("_")
    if team1 in old_to_new_id:
        team1 = old_to_new_id[team1]
    if team2 in old_to_new_id:
        team2 = old_to_new_id[team2]
    return "_".join([year, week, team1, team2])

team_performances = pd.read_csv("../../data/raw/team_performances.csv")
team_performances["game_id"] = team_performances.apply(clean_game_id, axis=1)

team_performances_game_id_to_row = defaultdict(list)
for index, row in team_performances.iterrows():
    team_performances_game_id_to_row[row.game_id].append(index)
    
for penalty in set(penalties.Phase_Penalty):
    team_performances[penalty] = 0

for index, row in penalties.iterrows():
    i = team_performances_game_id_to_row.get(row.Game_ID)
    if i is None:
        i = team_performances_game_id_to_row.get(row.Reverse_Game_ID)
    if i is None:
        continue
    j = i[0] if team_performances.iloc[i[0]].team_id == row.Team_ID else i[1]
    team_performances.loc[j, row.Phase_Penalty] += 1

team_performances

,game_id,team_id,pts,q1pts,q2pts,q3pts,q4pts,otpts,first_downs,rush-yds-tds,...,Off_Face_Mask,Off_Disqualification,Off_Offensive_12_On-field,Off_Offensive_Holding,Def_Lowering_the_Head_to_Initiate_Contact,Def_Illegal_Use_of_Hands,Off_Intentional_Grounding,Def_Defensive_Too_Many_Men_on_Field,Def_Illegal_Blindside_Block,Off_Unsportsmanlike_Conduct
0,2009_1_TEN_PIT,PIT,13,0,7,0,3,3.0,19,23-36-0,...,0,0,0,2,0,0,0,0,0,0
1,2009_1_TEN_PIT,TEN,10,0,7,0,3,0.0,18,25-86-0,...,0,0,0,0,0,0,0,0,0,0
2,2009_1_MIA_ATL,ATL,19,0,10,6,3,NaN,19,27-68-0,...,0,0,0,2,0,0,0,0,0,0
3,2009_1_MIA_ATL,MIA,7,0,0,0,7,NaN,16,22-96-0,...,0,0,0,2,0,1,0,0,0,0
4,2009_1_KC_BAL,BAL,38,10,0,7,21,NaN,32,41-198-2,...,0,0,0,3,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8121,2023_20_HOU_BAL,HOU,10,3,7,0,0,NaN,10,14-38-0,...,0,0,0,0,0,0,0,0,0,0
8122,2023_21_DET_SF,SF,34,0,7,17,10,NaN,23,33-155-3,...,0,0,0,0,0,0,0,0,0,0
8123,2023_21_DET_SF,DET,31,14,10,0,7,NaN,28,29-182-3,...,0,0,0,0,0,0,0,0,1,0
8124,2023_22_SF_KC,KC,25,0,3,10,6,6.0,24,30-130-0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
drives = pd.read_csv("../../data/processed/drives.csv")

game_id_to_index = {}
for index, row in drives.iterrows():
    if row.game_id not in game_id_to_index:
#         year, week, team1, team2 = row.game_id.split("_")
#         reverse_game_id = "_".join([year, week, team2, team1])
        game_id_to_index[row.game_id] = index
#         game_id_to_index[reverse_game_id] = index

for penalty in set(penalties.Phase_Penalty):
    drives[penalty] = 0
        
for index, row in penalties.iterrows():
    game_id = row.Game_ID
    drive_index = game_id_to_index.get(game_id)
    if drive_index is None:
        year, week, team1, team2 = row.Game_ID.split("_")
        game_id = "_".join([year, week, team2, team1])
        drive_index = game_id_to_index.get(game_id)
        if drive_index is None:
            continue
    while drive_index + 1 < len(drives) and drives.loc[drive_index + 1, "game_id"] == game_id and drives.loc[drive_index + 1, "time_left"] > row.Time_Left:
        drive_index += 1
    if drives.loc[drive_index, "time_left"] >= row.Time_Left:
        drives.loc[drive_index + 1, row.Phase_Penalty] += 1
        
drives

,game_id,team_id,num,quarter,time,los,plays,length,net_yds,result,...,Off_Face_Mask,Off_Disqualification,Off_Offensive_12_On-field,Off_Offensive_Holding,Def_Lowering_the_Head_to_Initiate_Contact,Def_Illegal_Use_of_Hands,Off_Intentional_Grounding,Def_Defensive_Too_Many_Men_on_Field,Def_Illegal_Blindside_Block,Off_Unsportsmanlike_Conduct
0,2009_1_TEN_PIT,PIT,1,1,15:00,PIT 42,3,1:44,2,Punt,...,0,0,0,0,0,0,0,0,0,0
1,2009_1_TEN_PIT,PIT,2,1,11:24,TEN 43,5,3:04,2,Punt,...,0,0,0,0,0,0,0,0,0,0
2,2009_1_TEN_PIT,PIT,3,1,6:44,PIT 27,3,1:55,-6,Punt,...,0,0,0,0,0,0,0,0,0,0
3,2009_1_TEN_PIT,PIT,4,1,1:38,PIT 21,3,1:32,3,Interception,...,0,0,0,0,0,0,0,0,0,0
4,2009_1_TEN_PIT,PIT,5,2,13:04,PIT 5,9,5:50,33,Punt,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92968,2023_22_SF_KC,SF,9,3,5:01,SFO 25,3,1:02,0,Punt,...,0,0,0,0,0,0,0,0,0,0
92969,2023_22_SF_KC,SF,10,3,2:28,SFO 25,12,6:06,75,Touchdown,...,0,0,0,0,0,0,0,0,0,0
92970,2023_22_SF_KC,SF,11,4,5:46,SFO 25,7,3:53,40,Field Goal,...,0,0,0,0,0,0,0,0,0,0
92971,2023_22_SF_KC,SF,12,4,0:03,SFO 25,1,0:03,-1,End of Half,...,0,0,0,0,0,0,0,0,0,0


In [6]:
penalties.to_csv("../../data/processed/penalties.csv")
team_performances.to_csv("../../data/processed/team_performances.csv")
drives.to_csv("../../data/processed/drives_v2.csv")